In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData, OpenCDP
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'SlatewiseAttention'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SlatewiseAttention(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention= torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        
        # let model attent to first padd token if slate is empty to avoid NaN gradients
        # (anyway they does not contrinute into metrics computation)
        key_padding_mask = batch['slates_mask'].clone()
        key_padding_mask[:,:, 0] = True 
        
        # repeating user embedding for each item embeding
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, shp[-2], 1).reshape(shp)
            ],
            dim = -1
        )
        
        # all recomendations in session are scored independently
        features = features.flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        
        # transforming back to sequence shape
        shp = list(shp)
        shp[-1] *= 2
        return self.out_layer(features.reshape(shp)).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SlatewiseAttention(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3614.74it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.6666666269302368, 'accuracy': 0.75}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.49
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 
Val update: epoch: 4 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.51
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SlatewiseAttention(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
   )
   (out_layer): Linear(in_features=64, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SlatewiseAttention(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SlatewiseAttention with svd embeddings
Test before learning: {'f1': 0.0, 'roc-auc': 0.5000449419021606, 'accuracy': 0.9026407599449158}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09961757063865662 | f1: 0.18118584156036377 | auc: 0.514898419380188 | treshold: 0.01
Test: accuracy: 0.09735925495624542 | f1: 0.17744280397891998 | auc: 0.5162686705589294 | 
Val update: epoch: 1 |accuracy: 0.09961757063865662 | f1: 0.18118584156036377 | auc: 0.5590634346008301 | treshold: 0.03
Test: accuracy: 0.09735925495624542 | f1: 0.17744280397891998 | auc: 0.5630252361297607 | 
Val update: epoch: 2 |accuracy: 0.15103010833263397 | f1: 0.18737085163593292 | auc: 0.568047046661377 | treshold: 0.03
Test: accuracy: 0.15901243686676025 | f1: 0.18467168509960175 | auc: 0.5702952742576599 | 
Val update: epoch: 3 |accuracy: 0.850188136100769 | f1: 0.2064853459596634 | auc: 0.5803830623626709 | treshold: 0.09999999999999999
Test: accuracy: 0.8504549860954285 | f1: 0.20918533205986023 | auc: 0.5837557911872864 | 
Val update: epoch: 4 |accuracy: 0.1452319324016571 | f1: 0.18781502544879913 | auc: 0.5984839797019958 | treshold: 0.06999999999999999
Test: ac

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09974092990159988 | f1: 0.18120616674423218 | auc: 0.5565532445907593 | treshold: 0.01
Test: accuracy: 0.09740528464317322 | f1: 0.1774502545595169 | auc: 0.562368631362915 | 
Val update: epoch: 1 |accuracy: 0.8137028217315674 | f1: 0.211577370762825 | auc: 0.6212557554244995 | treshold: 0.16
Test: accuracy: 0.8170505166053772 | f1: 0.2206680178642273 | auc: 0.6295391917228699 | 
Val update: epoch: 2 |accuracy: 0.8460245728492737 | f1: 0.22301766276359558 | auc: 0.6559537649154663 | treshold: 0.17
Test: accuracy: 0.8494422435760498 | f1: 0.22910119593143463 | auc: 0.6558631658554077 | 
Val update: epoch: 3 |accuracy: 0.7765544056892395 | f1: 0.27259036898612976 | auc: 0.6870778799057007 | treshold: 0.18000000000000002
Test: accuracy: 0.7824952602386475 | f1: 0.27348676323890686 | auc: 0.6818356513977051 | 
Val update: epoch: 4 |accuracy: 0.5993708372116089 | f1: 0.2574172616004944 | auc: 0.7045162916183472 | treshold: 0.16
Test: accuracy: 0.60849148035

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SlatewiseAttention(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SlatewiseAttention with neural embeddings
Test before learning: {'f1': 0.6059656739234924, 'roc-auc': 0.4744836688041687, 'accuracy': 0.5135050415992737}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7714168429374695 | f1: 0.8330300450325012 | auc: 0.8437093496322632 | treshold: 0.42000000000000004
Test: accuracy: 0.7773424983024597 | f1: 0.8386260271072388 | auc: 0.8437191247940063 | 
Val update: epoch: 1 |accuracy: 0.8119527697563171 | f1: 0.8562979102134705 | auc: 0.88646399974823 | treshold: 0.47000000000000003
Test: accuracy: 0.8149195909500122 | f1: 0.8595647811889648 | auc: 0.8862824440002441 | 
Val update: epoch: 2 |accuracy: 0.8252309560775757 | f1: 0.8680760860443115 | auc: 0.9041634202003479 | treshold: 0.46
Test: accuracy: 0.828920304775238 | f1: 0.871766984462738 | auc: 0.9027764797210693 | 
Val update: epoch: 3 |accuracy: 0.8343007564544678 | f1: 0.8750843405723572 | auc: 0.9129236936569214 | treshold: 0.49
Test: accuracy: 0.8361987471580505 | f1: 0.8773359656333923 | auc: 0.9113306999206543 | 
Val update: epoch: 4 |accuracy: 0.8389203548431396 | f1: 0.8757694363594055 | auc: 0.9148886203765869 | treshold: 0.47000000000000003
Test: ac

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7706912159919739 | f1: 0.8448002338409424 | auc: 0.881261944770813 | treshold: 0.44
Test: accuracy: 0.7761818170547485 | f1: 0.8490984439849854 | auc: 0.8790509700775146 | 
Val update: epoch: 1 |accuracy: 0.8186765313148499 | f1: 0.8637924194335938 | auc: 0.8938209414482117 | treshold: 0.45
Test: accuracy: 0.8205779194831848 | f1: 0.8659003973007202 | auc: 0.8920583724975586 | 
Val update: epoch: 3 |accuracy: 0.8254486322402954 | f1: 0.8703540563583374 | auc: 0.903662919998169 | treshold: 0.45
Test: accuracy: 0.8279531002044678 | f1: 0.8729759454727173 | auc: 0.9007708430290222 | 
Val update: epoch: 5 |accuracy: 0.8269965648651123 | f1: 0.8670865893363953 | auc: 0.905689001083374 | treshold: 0.47000000000000003
Test: accuracy: 0.8299117684364319 | f1: 0.8703027367591858 | auc: 0.9034794569015503 | 
Val update: epoch: 8 |accuracy: 0.8260049223899841 | f1: 0.874371349811554 | auc: 0.9137835502624512 | treshold: 0.46
Test: accuracy: 0.8281465172767639 | f

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7048565745353699 | f1: 0.7989521622657776 | auc: 0.7830830812454224 | treshold: 0.41000000000000003
Test: accuracy: 0.7118849158287048 | f1: 0.8049950003623962 | auc: 0.7776916027069092 | 
Val update: epoch: 1 |accuracy: 0.7304455041885376 | f1: 0.8088040947914124 | auc: 0.8025872111320496 | treshold: 0.42000000000000004
Test: accuracy: 0.7311570644378662 | f1: 0.8109569549560547 | auc: 0.797318696975708 | 
Val update: epoch: 2 |accuracy: 0.7446185946464539 | f1: 0.8131448030471802 | auc: 0.814402163028717 | treshold: 0.42000000000000004
Test: accuracy: 0.7455446720123291 | f1: 0.8153697848320007 | auc: 0.8106199502944946 | 
Val update: epoch: 3 |accuracy: 0.7535432577133179 | f1: 0.8121797442436218 | auc: 0.8219865560531616 | treshold: 0.45
Test: accuracy: 0.7544432282447815 | f1: 0.8147269487380981 | auc: 0.818434476852417 | 
Val update: epoch: 4 |accuracy: 0.7502055764198303 | f1: 0.823005199432373 | auc: 0.8309831619262695 | treshold: 0.43
Test: ac

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# OpenCDP

In [11]:
for group in ['cosmetics', 'multi']:
    for filename in os.listdir(pkl_path):    
        result = []
        if not filename.startswith(group):
            continue
        print(f"\n == {filename} ==")
        dataset = OpenCDP.load(os.path.join(pkl_path, filename))
        (
            train_loader, 
            val_loader,
            test_loader, 
            train_user_item_matrix, 
            train_num_items
        ) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=800)
    
        print(f"{len(dataset)} data points among {len(train_loader)} batches")
        for embeddings in ['neural', 'svd']:
            print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

            model = SlatewiseAttention(
                RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
                output_dim=1
            ).to(device)

            best_model, metrics = train(
                model, 
                train_loader, val_loader, test_loader, 
                device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
                silent=True
            )
            
            print(metrics)
            metrics['embeddings'] = embeddings
            result.append(metrics)
        pd.DataFrame(result).to_csv(f'results/{filename}_{experiment_name}.csv')
        del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items


 == cosmetics_10_8.pkl ==


AttributeError: 'OpenCDP' object has no attribute 'item_categorical'